In [1]:
%load_ext autoreload
%load_ext my_magics
%autoreload 2

In [2]:
# %%isortify
from model.violence_detection_model import ViolenceDetectionModel
import torch
from data_handling.video_dataset import VideoDataset
from data_handling.data_augmentation import VideoTransform
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
import utils

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Initialize the model.

In [4]:
vd_model = ViolenceDetectionModel(json_file="test.json")

Test the input-output of the model.

In [5]:
vd_model.eval()

videos = torch.randn(size=(2, 20, 3, 224, 224))
lengths = torch.tensor([20, 20])

outputs = vd_model(videos, lengths)

# for each video the model outputs a label between (0, 1)
assert outputs.size() == (2,)
assert all(0 < label < 1 for label in outputs)

print("Assertion pasted")

Assertion pasted


Sanity check for the model: test its capability of memorizing a small dataset -> if the model over fits, it is capable of learning.

In [6]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity


torch.cuda.empty_cache()
torch.cuda.synchronize()

vd_model.to(device)
vd_model.train()

transforms = VideoTransform(dataset="HockeyFights", json_file="test.json")
video_dataset = VideoDataset(dataset="HockeyFights", transformations=transforms, val=True)
data_loader = torch.utils.data.DataLoader(video_dataset, batch_size=10, collate_fn=utils.collate_fn_pad)

criterion = nn.BCELoss()  
optimizer = optim.Adam(vd_model.parameters(), lr=0.001)

for epoch in range(20):
    
    cumulating_loss = 0
    for batch in data_loader:
        videos, labels, lengths = batch 
        videos, labels = videos.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vd_model(videos, lengths)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        cumulating_loss += loss

        if device == "cuda":
            del videos, labels
            torch.cuda.empty_cache()

    print(f'Epoch {epoch}, Loss: {cumulating_loss / len(data_loader)}')


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.80 GiB. GPU 0 has a total capacity of 5.78 GiB of which 1.05 GiB is free. Including non-PyTorch memory, this process has 4.72 GiB memory in use. Of the allocated memory 4.60 GiB is allocated by PyTorch, and 23.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)